In [1]:
import pandas as pd
from datetime import datetime

def get_direct_link(shared_link):
    return 'https://drive.google.com/uc?export=download&id=' + shared_link.split('/')[-2]

link = "https://drive.google.com/file/d/1r7Onik0iXJ_2dyrjgdDv_8YVMlz8funh/view?usp=drive_link"
direct_link = get_direct_link(link)
wiki_data = pd.read_csv(direct_link, encoding='utf-8')

link = "https://drive.google.com/file/d/1sqMB2aRD2L77BsOiTDNjp-VzZX2NOEeR/view?usp=drive_link"
direct_link = get_direct_link(link)
odds_data = pd.read_csv(direct_link)

wiki_data['Date'] = pd.to_datetime(wiki_data['Date'])
wiki_data['date'] = wiki_data['Date']
wiki_data['pid'] = wiki_data.groupby('wiki_title').ngroup()

duplicates = wiki_data.duplicated(subset=['pid', 'Date'])
duplicates = duplicates[duplicates].index
wiki_data = wiki_data.drop(duplicates)

wiki_data = wiki_data.set_index(['pid', 'Date']).sort_index()


In [2]:
wiki_data

wiki_title    wiki    year  month   day  \
pid Date                                                           
0   2015-07-01  Agnieszka Radwańska  3349.0  2015.0    7.0   1.0   
    2015-07-02  Agnieszka Radwańska  2871.0  2015.0    7.0   2.0   
    2015-07-03  Agnieszka Radwańska  1905.0  2015.0    7.0   3.0   
    2015-07-04  Agnieszka Radwańska  3982.0  2015.0    7.0   4.0   
    2015-07-05  Agnieszka Radwańska  2066.0  2015.0    7.0   5.0   
...                             ...     ...     ...    ...   ...   
682 2023-07-26         Živa Falkner     5.0  2023.0    7.0  26.0   
    2023-07-27         Živa Falkner     2.0  2023.0    7.0  27.0   
    2023-07-28         Živa Falkner     2.0  2023.0    7.0  28.0   
    2023-07-29         Živa Falkner     3.0  2023.0    7.0  29.0   
    2023-07-30         Živa Falkner     1.0  2023.0    7.0  30.0   

                   original_names       date  
pid Date                                      
0   2015-07-01  ('Radwanska A.',) 2015-07-01  
    2015-07-02  ('Radwanska A.',) 2015-07-02  
    2015-07-03  ('Radwanska A.',) 2015-07-03  
    2015-07-04  ('Radwanska A.',) 2015-07-04  
    2015-07-05  ('Radwanska A.',) 2015-07-05  
...                           ...        ...  
682 2023-07-26    ('Falkner Z.',) 2023-07-26  
    2023-07-27    ('Falkner Z.',) 2023-07-27  
    2023-07-28    ('Falkner Z.',) 2023-07-28  
    2023-07-29    ('Falkner Z.',) 2023-07-29  
    2023-07-30    ('Falkner Z.',) 2023-07-30  

[2016216 rows x 7 columns]

In [3]:
wiki_data.sort_values(by=['pid', 'date'], inplace=True)

wiki_data['wiki_med365'] = wiki_data.groupby('pid')['wiki'].rolling(window=365, min_periods=0).median().reset_index(0, drop=True)
wiki_data['wiki_yesterday'] = wiki_data.groupby('pid')['wiki'].shift()

wiki_data = wiki_data.dropna(subset=['month', 'year', 'day'])

In [4]:
odds_data['date'] = pd.to_datetime(odds_data['Date'], format='%Y-%m-%d')
odds_data['year'] = odds_data['date'].dt.year
odds_data['month'] = odds_data['date'].dt.month
odds_data['day'] = odds_data['date'].dt.day

wiki_data['date'] = pd.to_datetime(wiki_data['date'], format='%d/%m/%Y')
wiki_data['year'] = wiki_data['date'].dt.year
wiki_data['month'] = wiki_data['date'].dt.month
wiki_data['day'] = wiki_data['date'].dt.day 

In [5]:
wiki_columns = ['date','wiki']+(list(wiki_data.columns[7:20]))

In [6]:
merged_data = pd.DataFrame()
j = -1
names = ''
for i, row in wiki_data.iterrows():
    names = row['original_names']
    winner_mask = odds_data['Winner'].apply(lambda x: x in names)
    temp_data = odds_data[winner_mask].copy()
    temp_data['original_names'] = names
    temp_data = temp_data.merge(pd.DataFrame([row[wiki_columns]]), on=['date'], how='inner')
    merged_data = pd.concat([merged_data,temp_data], ignore_index=True)
    if (i[0] % 50 ==0) & (i[0] > j):
        print(i[0])
        j = i[0]
        

for column in merged_data.columns:
    if column.startswith('wiki'):
        merged_data = merged_data.rename(columns={column: column + '_w'})


0
50
100
150
200
250
300
350
400
450
500
550
600
650


In [7]:
j = -1
names = ''
for i, row in wiki_data.iterrows():
    names = row['original_names']
    loser_mask = odds_data['Loser'].apply(lambda x: x in names)
    temp_data = odds_data[loser_mask].copy()
    temp_data['original_names'] = names
    temp_data = temp_data.merge(pd.DataFrame([row[wiki_columns]]), on=['date'], how='inner')
    merged_data = pd.concat([merged_data,temp_data], ignore_index=True)
    if (i[0] % 50 ==0) & (i[0] > j):
        print(i[0])
        j = i[0]
    
for column in merged_data.columns:
    if (column.startswith('wiki')) and not column.endswith('_w'):
        merged_data = merged_data.rename(columns={column: column + '_l'})

0
50
100
150
200
250
300
350
400
450
500
550
600
650


In [9]:
merged_data_l = merged_data[merged_data['wiki_w'].isnull()].reset_index()
merged_data_w = merged_data[merged_data['wiki_l'].isnull()]

combined_df = merged_data_w.combine_first(merged_data_l)

In [11]:
combined_df.to_csv('unclean_merged.csv', index=False)